In [1]:
! pip install telethon

In [2]:
import configparser
import json

In [3]:
from telethon.sync import TelegramClient
from telethon import connection

# для корректного переноса времени сообщений в json
from datetime import date, datetime

# классы для работы с каналами
from telethon.tl.functions.channels import GetParticipantsRequest
from telethon.tl.types import ChannelParticipantsSearch

# класс для работы с сообщениями
from telethon.tl.functions.messages import GetHistoryRequest

In [4]:
# Считываем учетные данные
config = configparser.ConfigParser()
config.read("config.ini")

['config.ini']

In [5]:
# Присваиваем значения внутренним переменным
api_id   = config['Telegram']['api_id']
api_hash = config['Telegram']['api_hash']
username = config['Telegram']['username']

In [6]:
print(api_id)
print(username)
print(api_hash)

13961036
sarana_a
464de18557f9ca2861e76e942bd342e5


In [7]:
client = TelegramClient(username, api_id, api_hash)

In [8]:
client.start()

<coroutine object AuthMethods._start at 0x000002CDFD4EFC40>

In [9]:
async def dump_all_messages(channel):
    #Записывает json-файл с информацией о всех сообщениях канала/чата"""
    offset_msg = 0    # номер записи, с которой начинается считывание
    limit_msg = 100   # максимальное число записей, передаваемых за один раз

    all_messages = []   # список всех сообщений
    total_messages = 0
    total_count_limit = 5000  # поменяйте это значение, если вам нужны не все сообщения
    class DateTimeEncoder(json.JSONEncoder):
        #Класс для сериализации записи дат в JSON'''
        def default(self, o):
            if isinstance(o, datetime):
                return o.isoformat()
            if isinstance(o, bytes):
                return list(o)
            return json.JSONEncoder.default(self, o)

    while True:
        history = await client(GetHistoryRequest(
            peer=channel,
            offset_id=offset_msg,
            offset_date=None, add_offset=0,
            limit=limit_msg, max_id=0, min_id=0,
            hash=0))
        if not history.messages:
            break
        messages = history.messages
        for message in messages:
            all_messages.append(message.to_dict().get('message', 'error'))
        offset_msg = messages[len(messages) - 1].id
        total_messages = len(all_messages)
        if total_count_limit != 0 and total_messages >= total_count_limit:
            break

    with open('channel_messages.json', 'w', encoding='utf8') as outfile:
        json.dump(all_messages, outfile, ensure_ascii=False, cls=DateTimeEncoder)



In [10]:
import nest_asyncio
nest_asyncio.apply()
from tqdm.notebook import tqdm

In [11]:
async def main():
    url = 't.me/GamCOVIDVac'
    channel = await client.get_entity(url)
    #await dump_all_participants(channel)
    await dump_all_messages(channel)


async with client:
    tqdm(client.loop.run_until_complete(main()))